In [24]:
import re
import pandas as pd
from pprint import pprint


def read_text_file_and_convert_to_list(filename):
    with open(filename, 'r', encoding="utf8") as file:
        content = file.read()
        split_content = content.split(';')
        split_content.pop()
        return split_content


class TextCleaner:
    """
    text cleaner class
    each page must complete all of the following conditions:
        at least x words per page
        at least x setences per page
        no word can have more that x chars
        if n sentences =<7:

            number of ok sentences> 2*number of suspected short sentences
        else:
            number of ok sentences> 4*number of suspected short sentences
        only hebrew and english charaters

        TODO:
            - no java script
            - no html
    """

    def __init__(
            self,
            separators_path='defult_seperators.txt',
            stopwords_path='stopwords',
            min_words_per_page=10,
            min_words_per_sentence_to_be_suspected_short_sentences=3,
            min_num_sentences=3,
            max_word_length=30

    ):
        self.separators = read_text_file_and_convert_to_list(separators_path)
        self.stuff_to_remove = read_text_file_and_convert_to_list(stopwords_path)
        self.min_words_per_page = min_words_per_page
        self.min_words_per_sentence_to_be_suspected_short_sentences = min_words_per_sentence_to_be_suspected_short_sentences
        self.min_num_sentences = min_num_sentences
        self.max_word_length = max_word_length

    def _split_by_page(self, txt):
        pattern = "".join(re.escape('<next_page>'))
        splitted_text_list = re.split(pattern, txt)
        return splitted_text_list

    def _split_setences(self, txt):
        pattern = "|".join(re.escape(seq) for seq in self.separators)
        splitted_text_list = re.split(pattern, txt)
        return splitted_text_list

    def _filter_hebrew_only(self, txt):
        filtered_text = ''.join([char for char in txt if
                                 ('\u0590' <= char <= '\u05FF') or ('\u0020' <= char <= '\u007F') or char in ['\n',
                                                                                                              '\t']])
        return filtered_text

    def _remove_stopwords_words(self, txt):
        for stuff in self.stuff_to_remove:
            txt = txt.replace(stuff, " ")
        return txt

    def _check_page(self, txt):
        txt = self._remove_stopwords_words(txt)
        senteces_df = pd.DataFrame({'text': self._split_setences(txt)})
        senteces_df = senteces_df.loc[(senteces_df['text'] is not None) & (senteces_df['text'] != '')]
        n_sentences_in_page = len(senteces_df)
        if n_sentences_in_page <= self.min_num_sentences:
            return '1'
        pattern = "|".join(re.escape(seq) for seq in ["\t", "\n", " ", ",", "."])
        senteces_df['splitted'] = senteces_df['text'].map(lambda x: re.split(pattern, x))
        senteces_df['splitted'] = senteces_df['splitted'].map(lambda x: [y for y in x if y not in ['', ' ', '.', ',']])
        senteces_df['len'] = senteces_df['splitted'].map(lambda x: len(x))
        n_words_in_page = senteces_df['len'].sum()
        if n_words_in_page <= self.min_words_per_page:
            return '2'
        # display(senteces_df )
        n_sentence_to_be_suspected_short_sentences_in_page = len(
            senteces_df.loc[senteces_df['len'] <= self.min_words_per_sentence_to_be_suspected_short_sentences])
        if n_sentences_in_page <= 7:
            if 2 * n_sentence_to_be_suspected_short_sentences_in_page >= n_sentences_in_page:
                return '3'
        else:
            if 4 * n_sentence_to_be_suspected_short_sentences_in_page >= n_sentences_in_page:
                return '3'
        senteces_with_words_df = senteces_df.loc[senteces_df['len']>0].copy()
        if len(senteces_with_words_df)>0:
            senteces_with_words_df['max_word_length'] = senteces_with_words_df['splitted'].map(lambda x: max([len(y) for y in x]))
        if senteces_with_words_df['max_word_length'].max() > self.max_word_length:
            return '4'
        return txt

    def clean_text(self, txt):
        txt = self._filter_hebrew_only(txt)
        pages_df = pd.DataFrame({'text': self._split_by_page(txt)})
        pages_df['cleaned'] = pages_df['text'].map(lambda x: self._check_page(x))
       # display(pages_df)
        cleaned_txt = "".join(pages_df['cleaned'].values.tolist())
        return cleaned_txt


if __name__ == '__main__':
    Cleaner = TextCleaner()
    filename = 'try.txt'
    with open(filename, 'r', encoding="utf8") as file:
        content = file.read()
    print(Cleaner.clean_text(content))







31333121 פרקא : דבר הנציב ועיקרי הדוח  5

 פרק א

דבר הנציב ועיקרי הדוח

מבוא
5 . מכוח  האמור  בסעיף 5 55  (א) לחוק  השיפוט  הצבאי, התשט"ו -5555  )להלן
החוק), הריני  מתכבד  להגיש  לשר הביטחון  ולוועדת  החוץ  והביטחון  של הכנסת
דין  וחשבון על  פעילות  נציב  קבילות  החיילים  בשנת  העבודה 51 50 .

5 . דוח זה גם מוגש לנשיא ה מדינה, לחברי הממשלה ולחברי הכנסת. כן הוא נשלח
לרמטכ"ל, לצמרת צה"ל, למפקדי היחידות ולמפקדים בכירים במילואים.

3 . כפי שהבעתי עמדתי בעבר נציב קבילות החיילים איננו תחליף למפקדים
ולחובתם לטפל כיאות בענייני פקודיהם ולהבטיח את זכויותיהם כבני -אדם
וכחיילים . אולם, כאש ר החייל חש שלא יהיה בכוחם של מפקדים לסייע לו או
שלא ימצא אוזן קשבת אצל מפקדיו, או שהוא חושש כי יבולע לו אם יתלונן
בפני המפקדים על עוול שנגרם לו   הוא עשוי לפנות לנציב, כגורם בלתי -תלוי,
על- מנת שעניינו ייבדק ללא משוא פנים. באשר על כן, ובתוקף התפקיד שהוטל
עלי י, הנני רואה עצמי בראש ובראשונה כנציגו של החייל הפונה .

0 . חובת מפקדי צה"ל לזכור כי הזכות הגדולה הכרוכה בנשיאת תפקיד פיקודי,
באשר הוא, כורכת בתוכה אחריות 

In [29]:

import os

def delete_files_with_substring(directory, substring):
    for filename in os.listdir(directory):
        if filename.endswith(".txt") and substring in filename:
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Deleted file: {filename}")

# Example usage
directory_path = r"C:\Users\MetroCat1\PycharmProjects\Pdf_Extraction_And_Cleaning\output"
substring_to_delete = "pont"
delete_files_with_substring(directory_path, substring_to_delete)